In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split

In [22]:
wandb.init(project='Kaggle House Prices Using MLP', entity='ma_heravi')

epoch,99
loss,13.8198
val_loss,15.81505
_runtime,66
_timestamp,1630419497
_step,199
best_val_loss,15.06219
best_epoch,90


epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇██
loss,████████████████████▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,████████████████████▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇████
_timestamp,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [23]:
train_data = pd.read_csv('train.csv')
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [24]:
train_data = train_data.replace(['A','FV', 'C (all)', 'I', 'RH', 'RL', 'RP', 'RM'],[0,1,2,3,4,5,6,7])
train_data = train_data.replace(['Pave','Grvl'],[0,1])
train_data = train_data.replace(['Lvl','Bnk', 'HLS', 'Low'],[0,1,2,3])
train_data = train_data.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'],[0,1,2,3])
train_data = train_data.replace(['Gtl','Mod', 'Sev'],[0,1,2])
train_data = train_data.replace(['Artery','Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],[0,1,2,3,4,5,6,7,8])
train_data = train_data.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],[0,1,2,3,4])
train_data = train_data.replace(['BrkTil','CBlock', 'PConc', 'Slab', 'Stone', 'Wood'],[0,1,2,3,4,5])
train_data = train_data.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'],[0,1,2,3,4,5])
train_data = train_data.fillna(0)

In [25]:
train_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,5,65.0,8450,0,0.0,Reg,0,0,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,5,80.0,9600,0,0.0,Reg,0,0,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,5,68.0,11250,0,0.0,IR1,0,0,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,5,60.0,9550,0,0.0,IR1,0,0,...,0,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,5,84.0,14260,0,0.0,IR1,0,0,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,5,62.0,7917,0,0.0,Reg,0,0,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,5,85.0,13175,0,0.0,Reg,0,0,...,0,0,MnPrv,0,0,2,2010,WD,Normal,210000
1457,1458,70,5,66.0,9042,0,0.0,Reg,0,0,...,0,0,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,5,68.0,9717,0,0.0,Reg,0,0,...,0,0,0,0,0,4,2010,WD,Normal,142125


In [26]:
target_data= train_data[['SalePrice']]
data = train_data.drop(columns=['Id', 'SalePrice'])

In [27]:
data_train = data[['OverallQual','LandSlope','Condition1','Condition2','BldgType','OverallCond','Foundation','Utilities','GarageType','LotArea','KitchenAbvGr','ScreenPorch','EnclosedPorch','LandContour','FullBath','HalfBath','BsmtFinSF1','GrLivArea','Fireplaces','TotalBsmtSF','ScreenPorch','WoodDeckSF','Street', 'Alley', 'MSZoning','1stFlrSF','YearRemodAdd','LotFrontage','YearBuilt','EnclosedPorch','GarageArea','GarageCars','OpenPorchSF','MSSubClass','TotRmsAbvGrd']]

In [28]:
data_train1 = data[['OverallQual','LotArea','MiscVal','KitchenAbvGr','ScreenPorch','EnclosedPorch','LandContour','2ndFlrSF','FullBath','HalfBath','BsmtFinSF1','PoolArea','BsmtFinSF2', 'MasVnrArea', 'BsmtUnfSF', 'LowQualFinSF','GrLivArea','Fireplaces','TotalBsmtSF','3SsnPorch','ScreenPorch','WoodDeckSF','Street', 'Alley', 'MSZoning','1stFlrSF','YearRemodAdd','LotFrontage','YearBuilt','EnclosedPorch','GarageArea','GarageCars','OpenPorchSF','MSSubClass','TotalBsmtSF']]

In [29]:
X_train, X_val, y_train, y_val = train_test_split(data_train, target_data, test_size=0.2, random_state=33)

In [30]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)

In [31]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)

# X_val = scaler.fit_transform(X_val)

In [32]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(8, activation='relu'),
#     tf.keras.layers.Dense(16, activation='relu'),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(8, activation='relu'),
#     tf.keras.layers.Dense(1, activation='linear'),
# ])

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, input_dim=35, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu'),
])

In [33]:
model.compile(loss=tf.keras.losses.mean_absolute_percentage_error,
              optimizer=tf.keras.optimizers.Adam(lr=0.001))

In [34]:
config = wandb.config
config.learning_rate = 0.001

In [35]:
output = model.fit(X_train, y_train,
                   validation_data=(X_val, y_val),
                   callbacks=[WandbCallback()],
                   epochs = 100)

Train on 1168 samples, validate on 292 samples
Epoch 1/100
1168/1168 [==============================] - 1s 623us/sample - loss: 57.3859 - val_loss: 26.7316
Epoch 2/100
1168/1168 [==============================] - 0s 144us/sample - loss: 24.5898 - val_loss: 21.6308
Epoch 3/100
1168/1168 [==============================] - 0s 106us/sample - loss: 21.9809 - val_loss: 20.0944
Epoch 4/100
1168/1168 [==============================] - 0s 106us/sample - loss: 19.5255 - val_loss: 19.4842
Epoch 5/100
1168/1168 [==============================] - 0s 106us/sample - loss: 18.1625 - val_loss: 19.1491
Epoch 6/100
1168/1168 [==============================] - 0s 111us/sample - loss: 18.1941 - val_loss: 17.2447
Epoch 7/100
1168/1168 [==============================] - 0s 82us/sample - loss: 18.3106 - val_loss: 21.8975
Epoch 8/100
1168/1168 [==============================] - 0s 77us/sample - loss: 17.0625 - val_loss: 17.5132
Epoch 9/100
1168/1168 [==============================] - 0s 147us/sample - loss: 16

In [36]:
test_data = pd.read_csv('test.csv')

In [37]:
test_data = test_data.replace(['A','FV', 'C (all)', 'I', 'RH', 'RL', 'RP', 'RM'],[0,1,2,3,4,5,6,7])
test_data = test_data.replace(['Pave','Grvl'],[0,1])
test_data = test_data.replace(['Lvl','Bnk', 'HLS', 'Low'],[0,1,2,3])
test_data = test_data.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'],[0,1,2,3])
test_data = test_data.replace(['Gtl','Mod', 'Sev'],[0,1,2])
test_data = test_data.replace(['Artery','Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],[0,1,2,3,4,5,6,7,8])
test_data = test_data.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],[0,1,2,3,4])
test_data = test_data.replace(['BrkTil','CBlock', 'PConc', 'Slab', 'Stone', 'Wood'],[0,1,2,3,4,5])
test_data = test_data.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'],[0,1,2,3,4,5])
test_data = test_data.fillna(0)

In [38]:
targer =  pd.read_csv('sample_submission.csv')
y_test = targer[['SalePrice']]

In [39]:
y_test = np.array(y_test)

In [40]:
x_test = test_data[['OverallQual','LandSlope','Condition1','Condition2','BldgType','OverallCond','Foundation','Utilities','GarageType','LotArea','KitchenAbvGr','ScreenPorch','EnclosedPorch','LandContour','FullBath','HalfBath','BsmtFinSF1','GrLivArea','Fireplaces','TotalBsmtSF','ScreenPorch','WoodDeckSF','Street', 'Alley', 'MSZoning','1stFlrSF','YearRemodAdd','LotFrontage','YearBuilt','EnclosedPorch','GarageArea','GarageCars','OpenPorchSF','MSSubClass','TotRmsAbvGrd']]

In [41]:
x_test = np.array(x_test)

In [42]:
# x_test = scaler.transform(x_test)

NameError: name 'scaler' is not defined

In [43]:
model.evaluate(x_test, y_test)

1459/1459 [==============================] - 0s 37us/sample - loss: 25.6465


25.64646328304466